<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/openai_agent_query_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# OpenAI代理工具描述过长的解决方法
在这个演示中，我们将演示一种解决方案，用于定义一个OpenAI工具，其描述超过OpenAI当前的1024个字符的限制。
为简单起见，我们将在`QueryPlanTool`笔记本示例的基础上进行演示。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4")

## 下载数据


In [ ]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-05-23 13:36:24--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  --.-KB/s    in 0.04s   

2024-05-23 13:36:24 (29.0 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-05-23 13:36:24--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

## 加载数据


In [ ]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

## 构建索引

我们在每个文档（3月、6月、9月）上构建一个向量索引/查询引擎。


In [ ]:
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)

In [ ]:
march_engine = march_index.as_query_engine(similarity_top_k=3, llm=llm)
june_engine = june_index.as_query_engine(similarity_top_k=3, llm=llm)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, llm=llm)

## 定义过长的查询计划

虽然一个 `QueryPlanTool` 可能由许多 `QueryEngineTools` 组成，但是当进行 OpenAI API 调用时，一个单独的 OpenAI 工具最终是由 `QueryPlanTool` 创建的。这个工具的描述以查询计划方法的一般说明开始，然后是每个组成部分 `QueryEngineTool` 的描述。

目前，每个 OpenAI 工具描述的最大长度为1024个字符。当你向你的 `QueryPlanTool` 添加更多的 `QueryEngineTools` 时，可能会超出这个限制。如果超出了限制，当 LlamaIndex 尝试构建 OpenAI 工具时，将会引发错误。

让我们通过一个夸张的例子来演示这种情况，我们将给每个查询引擎工具一个非常冗长和多余的描述。


In [ ]:
description_10q_general = """\Form 10-Q是美国证券交易委员会要求公开交易公司提交的季度报告，概述了公司当季的财务表现。报告包括未经审计的财务报表（损益表、资产负债表和现金流量表）和管理层讨论与分析（MD&A），管理层在其中解释重大变化和未来预期。10-Q还披露重大法律诉讼、风险因素更新，以及有关公司内部控制的信息。其主要目的是让投资者了解公司的财务状况和运营情况，从而做出明智的投资决策。"""description_10q_specific = (    "这份10-Q提供了Uber截至季度末的财务数据")

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import QueryPlanTool
from llama_index.core import get_response_synthesizer

In [ ]:
query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=f"{description_10q_general} {description_10q_specific} September 2022",
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=f"{description_10q_general} {description_10q_specific} June 2022",
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=f"{description_10q_general} {description_10q_specific} March 2022",
)

print(len(query_tool_sept.metadata.description))
print(len(query_tool_june.metadata.description))
print(len(query_tool_march.metadata.description))

730
725
726


从上面的打印语句中，我们可以看到当将这些工具组合成`QueryPlanTool`时，很容易超过1024的最大字符限制。


In [ ]:
query_engine_tools = [query_tool_sept, query_tool_june, query_tool_march]

response_synthesizer = get_response_synthesizer()
query_plan_tool = QueryPlanTool.from_defaults(
    query_engine_tools=query_engine_tools,
    response_synthesizer=response_synthesizer,
)

In [ ]:
openai_tool = query_plan_tool.metadata.to_openai_tool()

ValueError: Tool description exceeds maximum length of 1024 characters. Please shorten your description or move it to the prompt.

## 将工具描述移动到提示符

显而易见的解决方案是缩短工具描述本身，然而，如果有足够多的工具，最终仍然会超过字符限制。

更具可扩展性的解决方案是将工具描述移动到提示符。这解决了字符限制的问题，因为如果没有查询引擎工具的描述，查询计划描述将保持固定大小。当然，所选的LLM强加的令牌限制仍将限制工具描述，但这些限制远远大于1024个字符的限制。


将这些工具描述移动到提示中涉及两个步骤。首先，我们必须修改`QueryPlanTool`的元数据属性，以省略`QueryEngineTool`的描述，并对默认的查询规划说明进行轻微修改（告诉LLM在提示中查找工具名称和描述）。


In [ ]:
from llama_index.core.tools.types import ToolMetadataintroductory_tool_description_prefix = """\这是一个查询计划工具，它接收一个工具列表并执行这些工具的查询计划以回答查询。查询计划是查询节点的有向无环图（DAG）。给定工具名称列表和查询计划模式，您可以选择生成一个查询计划来回答问题。工具名称和描述将与查询一起给出。"""# 修改元数据以仅包括一般查询计划说明new_metadata = ToolMetadata(    introductory_tool_description_prefix,    query_plan_tool.metadata.name,    query_plan_tool.metadata.fn_schema,)query_plan_tool.metadata = new_metadataquery_plan_tool.metadata

ToolMetadata(description='This is a query plan tool that takes in a list of tools and executes a query plan over these tools to answer a query. The query plan is a DAG of query nodes.\n\nGiven a list of tool names and the query plan schema, you can choose to generate a query plan to answer a question.\n\nThe tool names and descriptions will be given alongside the query.\n', name='query_plan_tool', fn_schema=<class 'llama_index.core.tools.query_plan.QueryPlan'>, return_direct=False)

第二，我们必须将工具名称和描述与提出的查询连接起来。


In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

agent = OpenAIAgent.from_tools(
    [query_plan_tool],
    max_function_calls=10,
    llm=OpenAI(temperature=0, model="gpt-4-0613"),
    verbose=True,
)

query = "What were the risk factors in sept 2022?"

In [ ]:
# 重构连接的查询引擎工具描述tools_description = "\n\n".join(    [        f"工具名称：{tool.metadata.name}\n"        + f"工具描述：{tool.metadata.description} "        for tool in query_engine_tools    ])# 连接工具描述和查询query_planned_query = f"{tools_description}\n\n查询：{query}"query_planned_query

"Tool Name: sept_2022\nTool Description: A Form 10-Q is a quarterly report required by the SEC for publicly traded companies,\nproviding an overview of the company's financial performance for the quarter.\nIt includes unaudited financial statements (income statement, balance sheet,\nand cash flow statement) and the Management's Discussion and Analysis (MD&A),\nwhere management explains significant changes and future expectations.\nThe 10-Q also discloses significant legal proceedings, updates on risk factors,\nand information on the company's internal controls. Its primary purpose is to keep\ninvestors informed about the company's financial status and operations,\nenabling informed investment decisions. This 10-Q provides Uber quarterly financials ending September 2022 \n\nTool Name: june_2022\nTool Description: A Form 10-Q is a quarterly report required by the SEC for publicly traded companies,\nproviding an overview of the company's financial performance for the quarter.\nIt includes

In [ ]:
response = agent.query(query_planned_query)
response

Added user message to memory: Tool Name: sept_2022
Tool Description: A Form 10-Q is a quarterly report required by the SEC for publicly traded companies,
providing an overview of the company's financial performance for the quarter.
It includes unaudited financial statements (income statement, balance sheet,
and cash flow statement) and the Management's Discussion and Analysis (MD&A),
where management explains significant changes and future expectations.
The 10-Q also discloses significant legal proceedings, updates on risk factors,
and information on the company's internal controls. Its primary purpose is to keep
investors informed about the company's financial status and operations,
enabling informed investment decisions. This 10-Q provides Uber quarterly financials ending September 2022 

Tool Name: june_2022
Tool Description: A Form 10-Q is a quarterly report required by the SEC for publicly traded companies,
providing an overview of the company's financial performance for the quart

Response(response="The risk factors for Uber in September 2022 included:\n\n1. Failure to meet regulatory requirements related to climate change or to meet stated climate change commitments, which could impact costs, operations, brand, and reputation.\n2. The ongoing COVID-19 pandemic and responses to it were also a risk, as they had an adverse impact on business and operations, including reducing the demand for Mobility offerings globally and affecting travel behavior and demand.\n3. Catastrophic events such as disease, weather events, war, or terrorist attacks could also adversely impact the business, financial condition, and results of operation.\n4. Other risks included errors, bugs, or vulnerabilities in the platform's code or systems, inappropriate or controversial data practices, and the growing use of artificial intelligence.\n5. Climate change related physical and transition risks, such as market shifts toward electric vehicles and lower carbon business models, and risks relat